In [ ]:
import pandas as pd
import numpy as np
import requests
import searchtweets
from textblob import TextBlob
from afinn import Afinn
from datetime import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob, Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
import re
from nltk.corpus import stopwords
import nltk
import demoji

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
df = pd.read_csv("~/work/projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv")

In [ ]:
df.head()

# Nettoyage des données

In [ ]:
!pip install demoji

In [ ]:
# fonction pour la transformation du texte (retrait des caractères spéciaux, des émojis, de la ponctuation, tranformation des majuscules en minuscule, etc)
def transforme(texte):
    texte = texte.lower() # mettre les mots en minuscule
    #retirer les liens
    for item in re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+").findall(texte):
        texte=texte.replace(item, "")
    # enlever le retour à la ligne
    texte = texte.replace("\n", " ").replace("\r", "") 
    # supprimer ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\-\+\*\µ,\:\&\§]", " ", texte) 
    # retirer les hashtags #
    for item in re.compile("([#]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # retirer les personnes tagées
    for item in re.compile("([@]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # retire les mots contenant des chiffres
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    # retirer les emojis
    for item in demoji.findall(texte):
        texte=texte.replace(item,"")
    return texte
    

In [ ]:
pd.options.display.max_colwidth = 1000
df["texte1"]=df["text"].apply(lambda x: transforme(x))
df.tail()

In [ ]:
#transformer chaque phrase en liste de mots
df["liste_mots"] = df["texte1"].apply(lambda x: nltk.word_tokenize(x, language ='french'))

In [ ]:
df.head()

In [ ]:
# Enlever les stopwords
stop_words = set(stopwords.words('french'))
del df["stopword"]
df["mot_important"] = df["liste_mots"].apply(lambda x: [w for w in x if not w in stop_words])
df.tail()

# Calcul du label à partir de la polarité de Textblob

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
def tweet_polarite(text):
    return tb(text).sentiment[0]

def tweet_label(text):
    if tweet_polarite(text) > 0 :
        return 1
    if tweet_polarite(text) < 0 :
        return -1
    return 0

In [ ]:
df["score"]= df["texte1"].apply(tweet_polarite)

In [ ]:
df["label"]=df["texte1"].apply(tweet_label)
df.head()

# Visualisations

## Stats descriptives

## Avec Wordcloud

In [ ]:
df_cloud = df[df['label']==-1] 
# join tweets to a single string
words = ' '.join(df_cloud['texte1'])

wordcloud = WordCloud(stopwords=stop_words,
                      background_color='white',
                      width=2000,
                      height=1500, max_words=50
                     ).generate(words)

plt.figure(1,figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
df_cloud = df[df['label']==1] 
# join tweets to a single string
words = ' '.join(df_cloud['texte1'])

wordcloud = WordCloud(stopwords=stop_words,
                      background_color='white',
                      width=2000,
                      height=1500
                     ).generate(words)

plt.figure(1,figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Définition des métadonnées

In [ ]:
def decompte_hashtags(text):
    exp_match = re.compile("([#]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_hashtags"] = df_tweets["text"].apply(lambda x : decompte_hashtags(x))

In [ ]:
def decompte_tags(text):
    exp_match = re.compile("([@]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_tags"] = df_tweets["text"].apply(lambda x : decompte_tags(x))

In [ ]:
def decompte_mails(text):
    exp_match = re.compile("(\w+[@]\w+[.]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_mails"] = df_tweets["text"].apply(lambda x : decompte_mails(x))

In [ ]:
def decompte_exclamation(text):
    exp_match = re.compile("(\w?\s?[!])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_exclamation"] = df["text"].apply(lambda x : decompte_exclamation(x))

In [ ]:
def decompte_interrogation(text):
    exp_match = re.compile("(\w?\s?[?])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_interrogation"] = df_tweets["text"].apply(lambda x : decompte_interrogation(x))

In [ ]:
def decompte_mot_maj(text):
    exp_match = re.compile("([A-Z][A-Z]+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_maj"] = df_tweets["text"].apply(lambda x : decompte_mot_maj(x))

In [ ]:
df_tweets.head()

# Classification